## Part 2: Experiment and metrics design

The neighboring cities of Gotham and Metropolis have complementary circadian rhythms: on weekdays, Ultimate Gotham is most active at night, and Ultimate Metropolis is most active during the day. On weekends, there is reasonable activity in both cities.

However, a toll bridge, with a two way toll, between the two cities causes driver partners to tend to be exclusive to each city. The Ultimate managers of city operations for the two cities have proposed an experiment to encourage driver partners to be available in both cities, by reimbursing all toll costs.

1. What would you choose as the key measure of success of this experiment in encouraging driver partners to serve both cities, and why would you choose this metric?
2. Describe a practical experiment you would design to compare the effectiveness of the proposed change in relation to the key measure of success. Please provide details on:
   - how you will implement the experiment
   - what statistical test(s) you will conduct to verify the significance of the observation
   - how you would interpret the results and provide recommendations to the city operations team along with any caveats.

It sounds like the best time to run this experiement would be on the weekends, as this minimizes the difference of a driver's preference to work during the day or during the night. One assumption I see that this information provides is that drivers tend to pick one city over the other due to the toll bridge, but perhaps drivers have split themselves by city according to their ability or preference to drives nights over days (and vice versa). Therefore, focusing on the weekend is the best way to eliminate that potential bias. 

A key measure of success of this experiment would be first that the number of drivers making inter-city trips done on the would be higher, and second that the number of overall inter-city trips would be higher, too. 

Practical experiment: 
- The experiment would take place during weekends to minimize driver bias towards night or day drives.
- The statistical test would be an A/B test where certain drivers on the weekends would have their tolls reimbursed, and others would not. The drivers would be notified of this, but the toll-reimbursed drivers would have to be randomly selected from a pool of drivers with a history or working on the weekends ('history' meaning an established pattern of perhaps 3/4 weekends a month for a 3 month period within the last 6 months, for example).
- How test would be interpreted: The toll reimbursement has to be random or the trial is too self-selecting, so hopefully this bias would be minimized in the interpretation of the test. From the random group selected as per the parameters outlined above ('history' of working weekends), an analysis of the number of inter-city trips would be assessed. The drivers would have to know if they were in the group or not, because fundamentally this experiment is testing the power of toll-reimbursement as a motivator for working between two cities, over sticking to just one city. Statistically, we would consider a one-sided p-value of 0.05 as an indicator that toll reimbursements are worth potentially investing in. 
- The maain caveats were described above, and a spearate experiment may be conducted to include weekdays if a way to account for the difference in rider time can be accounted for. Another risk may be that if a weekend driver is not selected to have their tolls reimbursed, they are now dis-incentivized to make inter-city trips. A pre-experiment step to prevent this may be to send out a survey asking if drivers were reimbursed for tolls on weekend trips, how much more inclined would they be to make inter-city trips? It would also be useful to poll them for other forms of motivation to make inter-city trips, but that is outside the scope of this request. 

If the results of this experiment indicate that toll reimbursement effectively increases the number of inter-city trips on the weekends for drivers, experiments will be rolled out for weekday drivers. 


## Part 3: Predictive Modeling

1. Perform any cleaning, exploratory analysis, and/or visualizations to use the provided data for this analysis (a few sentences/plots describing your approach will suffice). What fraction of the observed users were retained?
2. Build a predictive model to help Ultimate determine whether or not a user will be active in their 6th month on the system. Discuss why you chose your approach, what alternatives you considered, and any concerns you have. How valid is your model? Include any key indicators of model performance.
3. Briefly discuss how Ultimate might leverage the insights gained from the model to improve its long term rider retention (again, a few sentences will suffice).

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data Wrangling

In [3]:
df = pd.read_json('ultimate_data_challenge.json')
df.head()

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver
0,King's Landing,4,2014-01-25,4.7,1.10,2014-06-17,iPhone,15.4,True,46.2,3.67,5.0
1,Astapor,0,2014-01-29,5.0,1.00,2014-05-05,Android,0.0,False,50.0,8.26,5.0
2,Astapor,3,2014-01-06,4.3,1.00,2014-01-07,iPhone,0.0,False,100.0,0.77,5.0
3,King's Landing,9,2014-01-10,4.6,1.14,2014-06-29,iPhone,20.0,True,80.0,2.36,4.9
4,Winterfell,14,2014-01-27,4.4,1.19,2014-03-15,Android,11.8,False,82.4,3.13,4.9


In [4]:
df.isna().sum()

city                         0
trips_in_first_30_days       0
signup_date                  0
avg_rating_of_driver      8122
avg_surge                    0
last_trip_date               0
phone                      396
surge_pct                    0
ultimate_black_user          0
weekday_pct                  0
avg_dist                     0
avg_rating_by_driver       201
dtype: int64